# Webscrape NFL Games, Weather

### Set-up

In [49]:
# Import packages
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time

# Directory
WORKING_DIRECTORY = "/Users/peterchristenson/Desktop/Projects/NFL Weather Analysis/" 

### Set relevant web scraping info

In [52]:
# Seasons to check
seasons = list(range(2000, 2023))

# Main webpage
webpage_main = "https://www.profootballarchives.com/"

### Web scrape

In [ ]:
game_df = pd.DataFrame()
for season in seasons:
    game_yr_df = pd.DataFrame()
    try:
        webpage = webpage_main + str(season) + "nfl-boxscores.html"
        response = requests.get(webpage)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Get links to games
        links_raw = soup.find_all("a")
        links = []
        for link in links_raw:
            if "nflboxscores" in link["href"]:
                links.append(link["href"])
            
        print(links[0])        
        # Loop through each game, pull attributes
        for link in links:
            try:
                webpage = "https://www.profootballarchives.com" + link
                response = requests.get(webpage)
                soup = BeautifulSoup(response.text, 'html.parser')
                
                teams = soup.find("title") # Teams playing game
                date = None
                location = None
                venue = None
                weather = None
                for attr in soup.find_all("td"):
                    attr_text = attr.get_text()
                    if "Date" in attr_text:
                        date = attr_text
                    if "Location" in attr_text:
                        location = attr_text
                    if "Venue" in attr_text:
                        venue = attr_text
                    if "Weather:" in attr_text:
                        weather = attr_text
                        
                # Points
                points = soup.find_all("table")
                score_table = [pt for pt in points if "Score By Quarters" in pt.get_text()]
                score_table = score_table[0].find_all("td", {"style":"text-align:right"})
                team_1_pts = score_table[int(len(score_table)/2) - 1].get_text()
                team_2_pts = score_table[len(score_table) - 1].get_text()
                
                # Add to main dataframe
                df = pd.DataFrame(
                    {'teams':[teams],
                    'team 1 pts':[team_1_pts],
                    'team 2 pts':[team_2_pts],
                    'date':[date],
                    'location':[location],
                    'venue':[venue],
                    'weather':[weather]
                    }
                )
                
                game_df = pd.concat([game_df, df])
                game_yr_df = pd.concat([game_yr_df, df])
                    
                time.sleep(0.1) # Time buffer between webpage calls
                print(link)
            except:
                continue
    except:
        continue
    game_yr_df.to_csv(WORKING_DIRECTORY + "intermediate/" + str(season) + ".csv")